In [28]:
!pip install spacy-transformers
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [30]:
!git clone https://github.com/tulee3474/AI_Resume_Builder

fatal: destination path 'AI_Resume_Builder' already exists and is not an empty directory.


In [31]:
%cd /content/AI_Resume_Builder/
!git pull

/content/AI_Resume_Builder
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (2/2), done.
remote: Total 42 (delta 2), reused 2 (delta 2), pack-reused 40
Unpacking objects: 100% (42/42), done.
From https://github.com/tulee3474/AI_Resume_Builder
   a06118d..27988d5  main       -> origin/main
Updating a06118d..27988d5
Fast-forward
 resume/51.txt |  73 +++++++++++++-----------------------------
 resume/52.txt |  83 ++++++++++++++---------------------------------
 resume/53.txt |  28 ++++++++--------
 resume/54.txt |  51 +++++++++++------------------
 resume/55.txt | 101 +++++++++++++++++++++++++---------------------------------
 resume/56.txt |  35 ++++++++++++++++++++
 resume/57.txt |  36 +++++++++++++++++++++
 resume/58.txt |  55 ++++++++++++++++++++++++++++++++
 resume/59.txt |  39 +++++++++++++++++++++++
 resume/60.txt |  42 ++++++++++++++++++++++++
 resume/61.txt |  27 ++++++++++++++++
 resume/62.txt |  17 ++++++++++
 resume/63.txt |  19 +++++++++++
 resume/64.txt 

In [32]:
import json
cv_data=json.load(open('/content/AI_Resume_Builder/Job_description_annotated/mergerd jd.json','r'))

In [33]:
!python -m spacy init fill-config /content/AI_Resume_Builder/base_config.cfg /content/AI_Resume_Builder/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/AI_Resume_Builder/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [34]:
def get_spacy_doc(file, data):
  nlp=spacy.blank('en')
  db=DocBin()

  for item in data:
    text = item['text']
    doc=nlp.make_doc(text)
    annot=item['label']

    
    ents=[]
    entity_indices=[]

    for start, end, label in annot:
      skip_entity=False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
         continue
        
      entity_indices = entity_indices + list(range(start, end))

      try:
        span=doc.char_span(start,end,label=label,alignment_mode='contract')
      except:
        continue
        
      if span is None:
        err_data=str([start,end]) + "     " + str(text) + "\n"
        file.write(err_data)
        
      else:
        ents.append(span)

    try:
        doc.ents=ents
        db.add(doc)
    except:
      pass


  return db

In [35]:
from sklearn.model_selection import train_test_split
train, test=train_test_split(cv_data,test_size=0.3)

In [36]:
len(train), len(test)

(111, 48)

In [37]:
cv_data[0]

{'id': 19,
 'label': [[0, 5, 'company'],
  [337, 357, 'project'],
  [362, 418, 'project'],
  [421, 459, 'job'],
  [475, 481, 'required skills'],
  [483, 488, 'required skills'],
  [489, 500, 'required skills'],
  [505, 513, 'required skills'],
  [642, 690, 'project'],
  [695, 772, 'project'],
  [809, 847, 'benefit'],
  [887, 927, 'benefit'],
  [989, 1057, 'benefit'],
  [1160, 1218, 'benefit'],
  [1464, 1532, 'benefit']],
 'text': "Yapz is a cutting edge, growth-stage startup that allows you to find, connect, and co-host events with clubs in your local area. In addition to their schooling, social, family and work lives, college students partake in extracurricular clubs on campus. To allocate enough time to make those clubs successful is extremely difficult. Yapz makes running a club and finding partner clubs for events easy and time efficient.\n\nWe are looking for a Backend Developer who works with Python, flask, SQLAlchemy and Postgres. The developer should also have a practical under

In [38]:
file=open('error.txt','w')
db=get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db=get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

In [39]:
db.tokens

[array([[15359919646060204151,  2672670766848947069,                    0,
         ...,                    0,                    0,
                            0],
        [18072887903199230677, 18072887903199230677,                    0,
         ...,                    0,                    0,
                            0],
        [12363111293408059666, 12363111293408059666,                    0,
         ...,                    0,                    0,
                            0],
        ...,
        [11532473245541075862, 11532473245541075862,                    0,
         ...,                    0,                    0,
                            0],
        [  962983613142996970,   962983613142996970,                    0,
         ...,                    0,                    0,
                            0],
        [ 9692231336188541085, 13055779130471031426,                    0,
         ...,                    0,                    0,
                            0

In [40]:
!python -m spacy train /content/AI_Resume_Builder/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-08-08 03:33:06,029] [INFO] Set up nlp object from config
[2022-08-08 03:33:06,039] [INFO] Pipeline: ['transformer', 'ner']
[2022-08-08 03:33:06,044] [INFO] Created vocabulary
[2022-08-08 03:33:06,045] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a 

In [41]:
nlp=spacy.load('/content/AI_Resume_Builder/output/model-best') 

In [42]:
doc=nlp('Seasoned and independent Front End Developer with 5 years of experience in blending the art of design with skill of programming to deliver an immersive and engaging user experience through efficient website development, proactive feature optimization, and relentless debugging.')
for ent in doc.ents:
  print(ent.text, "    ->>>> ", ent.label_)

Front End Developer     ->>>>  job
5 years of experience     ->>>>  required year


In [43]:
!pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
import sys, fitz

In [45]:
for x in range(1,11):
  fname='/content/AI_Resume_Builder/jd test/'+str(x)+'.pdf'
  doc=fitz.open(fname)
  print('job description num '+str(x)+' test')
  text=" "
  for page in doc:
    text=text+str(page.get_text())
  text=text.strip()
  text=' '.join(text.split())
  doc=nlp(text)
  for ent in doc.ents:
    print(ent.text, " ->>>>>> ", ent.label_)
  print('\n')

job description num 1 test
Django Developer  ->>>>>>  job
Exposure to Scrum or Agile software development processes  ->>>>>>  required skills
Self managing,  ->>>>>>  required skills
comfortable with remote working Desire to learn Basic project management Required  ->>>>>>  required skills
1+ years  ->>>>>>  required year
Python software development  ->>>>>>  required skills
2+ years  ->>>>>>  required year
XML  ->>>>>>  required skills
REST  ->>>>>>  required skills
Web Frameworks  ->>>>>>  required skills
template languages  ->>>>>>  required skills
GIT  ->>>>>>  required skills
MySQL Database Message based  ->>>>>>  required skills
Asynchronous processing systems  ->>>>>>  required skills
Flask  ->>>>>>  required skills
Django  ->>>>>>  required skills
Frontend skills  ->>>>>>  required skills
HTML  ->>>>>>  required skills
CSS  ->>>>>>  required skills
Javascript  ->>>>>>  required skills
JQuery  ->>>>>>  required skills
Bootstrap  ->>>>>>  required skills
B.Tech/B.E. - Computers, 